In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import numpy as np 
import pandas as pd 
import os
import re

In [3]:
toy_rev = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Aspect-Based Sentiment Analysis/Scrapped_Car_Reviews_Toyota.csv',engine='python',index_col=False)
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating
0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5.000
1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5.000
2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5.000
3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",4.875
4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5.000


In [4]:
toy_rev['review']=toy_rev['Review_Title']+toy_rev['Review']
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review
0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5.000,"great vehicle, Toyota best design ever. thank ..."
1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5.000,"my 4th previa, best van ever made! 1st 95 went..."
2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5.000,Mom's Taxi Babies Ride Sold 86 Toyota Van 285K...
3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",4.875,My Favorite Van Ever I have owned lots of vans...
4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5.000,Best Minivan ever My 1997 AWD Previa is the th...


In [5]:
# Using spaCy for dependency parsing which forms the crux of aspect extraction

import spacy
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [6]:
# Using spaCy's awesome displacy module to show the dependency relations

txt = 'Great car and has long range'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

# AMOD - adjectival modifier
# An adjectival modifier of a Noun is any adjectival phrase that serves to modify the meaning of the Noun
# ex - 'Great <--amod-- Car', 'Long <--amod-- range'

In [7]:
txt = 'Drives well has great handling'
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [8]:
txt =  "wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [9]:
txt =  "not wonderful to drive the camry "
doc = nlp(txt)
spacy.displacy.render(doc,style='dep',jupyter=True)

In [10]:
# Reason for using competitor name list is to remove potential misleading aspects-sentiments, since we are interested to
#acquire aspect info about Toyota and not any other brand.


competitors = ['Chevy','chevy','Ford','ford','Nissan','nissan','Honda','honda','Chevrolet','chevrolet','Volkswagen','volkswagen',
               'benz','Benz','Mercedes','mercedes','subaru','Subaru','VW']


In [11]:
aspect_terms = []
comp_terms = []
easpect_terms = []
ecomp_terms = []
enemy = []
for x in tqdm(range(len(toy_rev['review']))):
    amod_pairs = []
    advmod_pairs = []
    compound_pairs = []
    xcomp_pairs = []
    neg_pairs = []
    eamod_pairs = []
    eadvmod_pairs = []
    ecompound_pairs = []
    eneg_pairs = []
    excomp_pairs = []
    enemlist = []
    if len(str(toy_rev['review'][x])) != 0:
        lines = str(toy_rev['review'][x]).replace('*',' ').replace('-',' ').replace('so ',' ').replace('be ',' ').replace('are ',' ').replace('just ',' ').replace('get ','').replace('were ',' ').replace('When ','').replace('when ','').replace('again ',' ').replace('where ','').replace('how ',' ').replace('has ',' ').replace('Here ',' ').replace('here ',' ').replace('now ',' ').replace('see ',' ').replace('why ',' ').split('.')       
        for line in lines:
            enem_list = []
            for eny in competitors:
                enem = re.search(eny,line)
                if enem is not None:
                    enem_list.append(enem.group())
            if len(enem_list)==0:
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                compound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                amod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        amod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    amod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                advmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                neg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                advmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j,h in zip(token.rights,token.lefts):
                            if j.dep_ is 'xcomp' and h.dep_ is not 'neg':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        xcomp_pairs.append(token.text+' '+k.text+' '+j.text)
                            elif j.dep_ is 'xcomp' and h.dep_ is 'neg':
                                if k.dep_ is 'aux':
                                        neg_pairs.append(h.text +' '+token.text+' '+k.text+' '+j.text)
            
            else:
                enemlist.append(enem_list)
                doc = nlp(line)
                str1=''
                str2=''
                for token in doc:
                    if token.pos_ is 'NOUN':
                        for j in token.lefts:
                            if j.dep_ == 'compound':
                                ecompound_pairs.append((j.text+' '+token.text,token.text))
                            if j.dep_ is 'amod' and j.pos_ is 'ADJ': #primary condition
                                str1 = j.text+' '+token.text
                                eamod_pairs.append(j.text+' '+token.text)
                                for k in j.lefts:
                                    if k.dep_ is 'advmod': #secondary condition to get adjective of adjectives
                                        str2 = k.text+' '+j.text+' '+token.text
                                        eamod_pairs.append(k.text+' '+j.text+' '+token.text)
                                mtch = re.search(re.escape(str1),re.escape(str2))
                                if mtch is not None:
                                    eamod_pairs.remove(str1)
                    if token.pos_ is 'VERB':
                        for j in token.lefts:
                            if j.dep_ is 'advmod' and j.pos_ is 'ADV':
                                eadvmod_pairs.append(j.text+' '+token.text)
                            if j.dep_ is 'neg' and j.pos_ is 'ADV':
                                eneg_pairs.append(j.text+' '+token.text)
                        for j in token.rights:
                            if j.dep_ is 'advmod'and j.pos_ is 'ADV':
                                eadvmod_pairs.append(token.text+' '+j.text)
                    if token.pos_ is 'ADJ':
                        for j in token.rights:
                            if j.dep_ is 'xcomp':
                                for k in j.lefts:
                                    if k.dep_ is 'aux':
                                        excomp_pairs.append(token.text+' '+k.text+' '+j.text)
        pairs = list(set(amod_pairs+advmod_pairs+neg_pairs+xcomp_pairs))
        epairs = list(set(eamod_pairs+eadvmod_pairs+eneg_pairs+excomp_pairs))
        for i in range(len(pairs)):
            if len(compound_pairs)!=0:
                for comp in compound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(pairs[i]))
                    if mtch is not None:
                        pairs[i] = pairs[i].replace(mtch.group(),comp[0])
        for i in range(len(epairs)):
            if len(ecompound_pairs)!=0:
                for comp in ecompound_pairs:
                    mtch = re.search(re.escape(comp[1]),re.escape(epairs[i]))
                    if mtch is not None:
                        epairs[i] = epairs[i].replace(mtch.group(),comp[0])
            
    aspect_terms.append(pairs)
    comp_terms.append(compound_pairs)
    easpect_terms.append(epairs)
    ecomp_terms.append(ecompound_pairs)
    enemy.append(enemlist)
toy_rev['compound_nouns'] = comp_terms
toy_rev['aspect_keywords'] = aspect_terms
toy_rev['competition'] = enemy
toy_rev['competition_comp_nouns'] = ecomp_terms
toy_rev['competition_aspects'] = easpect_terms
toy_rev.head()
 

100%|██████████| 22702/22702 [22:06<00:00, 17.11it/s]


,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects
0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5.000,"great vehicle, Toyota best design ever. thank ...",[],"[great vehicle, best design]",[],[],[]
1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5.000,"my 4th previa, best van ever made! 1st 95 went...","[(Middle bench, bench), (captain chairs, chair...","[Easy to fix, Walked away, middle captain, rel...",[],[],[]
2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5.000,Mom's Taxi Babies Ride Sold 86 Toyota Van 285K...,"[(K miles, miles), (2nd kid, kid), (weekend du...","[remote bat, middle seat, change later, 1st ba...",[],[],[]
3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",4.875,My Favorite Van Ever I have owned lots of vans...,"[(Build quality, quality), (Fuel mileage, mile...","[Ever owned, mid engine, ever owned, best van]",[],[],[]
4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5.000,Best Minivan ever My 1997 AWD Previa is the th...,"[(K mi, mi), (gas mileage, mileage)]","[great K gas mileage, third one, reasonable re...",[],[],[]


In [13]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 3.4MB/s 


In [14]:
# We use vaderSentiment for sentiment analysis because of it's speed and simplicity. 
# It offers 3 types of polarity - positive, negative and neutral.

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [21]:
import operator
sentiment = []
for i in range(len(toy_rev)):
    score_dict={'pos':0,'neg':0,'neu':0,'compound':0}
    if len(toy_rev['aspect_keywords'][i])!=0: 
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
            score_dict['neu'] += sent['neu']
            score_dict['compound'] += sent['compound']
        sentiment.append(max(score_dict.items(), key=operator.itemgetter(1))[0])
    else:
        sentiment.append('NaN')
toy_rev['sentiment'] = sentiment
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,sentiment_score
0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5.000,"great vehicle, Toyota best design ever. thank ...",[],"[great vehicle, best design]",[],[],[],pos,1.612
1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5.000,"my 4th previa, best van ever made! 1st 95 went...","[(Middle bench, bench), (captain chairs, chair...","[Easy to fix, Walked away, middle captain, rel...",[],[],[],neu,13.145
2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5.000,Mom's Taxi Babies Ride Sold 86 Toyota Van 285K...,"[(K miles, miles), (2nd kid, kid), (weekend du...","[remote bat, middle seat, change later, 1st ba...",[],[],[],neu,7.4
3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",4.875,My Favorite Van Ever I have owned lots of vans...,"[(Build quality, quality), (Fuel mileage, mile...","[Ever owned, mid engine, ever owned, best van]",[],[],[],neu,3.192
4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5.000,Best Minivan ever My 1997 AWD Previa is the th...,"[(K mi, mi), (gas mileage, mileage)]","[great K gas mileage, third one, reasonable re...",[],[],[],neu,5.423


In [22]:
sentiment_score = []
for i in range(len(toy_rev)):
    score_dict={'pos':0,'neg':0,'neu':0,'compound':0}
    if len(toy_rev['aspect_keywords'][i])!=0: 
        for aspects in toy_rev['aspect_keywords'][i]:
            sent = analyser.polarity_scores(aspects)
            score_dict['neg'] += sent['neg']
            score_dict['pos'] += sent['pos']
            score_dict['neu'] += sent['neu']
            score_dict['compound'] += sent['compound']
        sentiment_score.append(max(score_dict.values()))
    else:
        sentiment_score.append('NaN')
toy_rev['sentiment_score'] = sentiment_score
toy_rev.head()

,Unnamed: 0,Review_Date,Author_Name,Vehicle_Title,Review_Title,Review,Rating,review,compound_nouns,aspect_keywords,competition,competition_comp_nouns,competition_aspects,sentiment,sentiment_score
0,0,on 02/02/17 19:53 PM (PST),Ricardo,1997 Toyota Previa Minivan LE 3dr Minivan,"great vehicle, Toyota best design ever. thank you","there is no way back, enjoy what you have .",5.000,"great vehicle, Toyota best design ever. thank ...",[],"[great vehicle, best design]",[],[],[],pos,1.612
1,1,on 12/17/16 16:40 PM (PST),matt,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,"my 4th previa, best van ever made!",1st 95 went over 300k before being totalled b...,5.000,"my 4th previa, best van ever made! 1st 95 went...","[(Middle bench, bench), (captain chairs, chair...","[Easy to fix, Walked away, middle captain, rel...",[],[],[],neu,13.145
2,2,on 04/14/10 07:43 AM (PDT),Joel G,1997 Toyota Previa Minivan LE 3dr Minivan,Mom's Taxi Babies Ride,Sold 86 Toyota Van 285K miles to be replaced ...,5.000,Mom's Taxi Babies Ride Sold 86 Toyota Van 285K...,"[(K miles, miles), (2nd kid, kid), (weekend du...","[remote bat, middle seat, change later, 1st ba...",[],[],[],neu,7.4
3,3,on 11/12/08 17:31 PM (PST),Dennis,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,My Favorite Van Ever,"I have owned lots of vans, and the Previa is ...",4.875,My Favorite Van Ever I have owned lots of vans...,"[(Build quality, quality), (Fuel mileage, mile...","[Ever owned, mid engine, ever owned, best van]",[],[],[],neu,3.192
4,4,on 04/14/08 22:47 PM (PDT),Alf Skrastins,1997 Toyota Previa Minivan LE All-Trac 3dr Min...,Best Minivan ever,My 1997 AWD Previa is the third one that I ha...,5.000,Best Minivan ever My 1997 AWD Previa is the th...,"[(K mi, mi), (gas mileage, mileage)]","[great K gas mileage, third one, reasonable re...",[],[],[],neu,5.423
